In [2]:
%conda install numpy
%conda install pandas
%conda install sklearn
%conda install matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.7.22          |  py310h06a4308_0         153 KB
    ------------------------------------------------------------
                                           Total:         276 KB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2023.7.2~ --> pkgs/main::ca-certificates-2023.08.22-h06a4308_0 

The following packages will be

In [276]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [56]:
bucket='205-datasets'
data_key = 'used-cars/train.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
raw = pd.read_csv(data_location)

In [233]:
# initial cleanup
cars = raw.copy()
cars.drop("Unnamed: 0", axis=1, inplace=True)
cars.columns = [x.lower() for x in cars.columns]

In [283]:
cars.head()

,name,location,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,new_price,price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [ ]:
# TODO parse make from name

In [378]:
flat = []
for words in cars["name"].str.split(" ").values:
    flat +=  words

In [382]:
pd.Series(flat).value_counts().head(50)

Maruti           1132
Hyundai          1057
Honda             607
AT                540
Diesel            425
1.2               415
Toyota            395
TDI               387
Swift             337
MT                337
Mercedes-Benz     316
Volkswagen        314
i10               307
Plus              304
Ford              295
1.5               295
S                 285
1.6               281
City              270
Mahindra          269
BMW               267
i20               261
CRDi              259
SX                253
Audi              235
VXI               225
Sportz            211
2.0               194
Series            189
Tata              181
Highline          180
Dzire             176
Magna             175
CDI               174
V                 173
Skoda             172
Option            171
Verna             170
1.4               166
Petrol            165
Innova            164
VDI               158
Grand             158
i                 156
VX                155
Polo      

In [338]:
# will be passed a dataframe and should return a dataframe
def remove_units(df):
    return df.apply(lambda x: x.str.split(" ").str[0].astype("float"))

def convert_years(df):
    return df.apply(lambda x: 2019 - x)

def bin_owner(df):
    return df.applymap(lambda x: "Third" if x not in ["First", "Second"] else x)

def ordinal_owner(df):
    owner_map = {
        "First": 1,
        "Second": 2,
        "Third": 3
    }
    
    return df.applymap(lambda x: owner_map[x])

In [344]:
owner_pipeline = Pipeline(
    [
        ('high_val_bin', FunctionTransformer(bin_owner)),
        ('ordinal_encode', FunctionTransformer(ordinal_owner))
    ]
)

ct = ColumnTransformer(
    [
         # seems to only work when passed a list of column names and expects function to take and return a dataframe
        ("owner_pipeline", owner_pipeline, ["owner_type"]),
        ("one_hot", OneHotEncoder(handle_unknown="ignore"), ["location", "fuel_type"]),
        ("ordinal", OrdinalEncoder(), ["transmission"]),
        ("remove_units", FunctionTransformer(remove_units), ["mileage", "engine", "power"]),
        ("conver_years", FunctionTransformer(convert_years), ["year"]),
    ],
    # remainder='passthrough'
)

In [358]:
pipe = Pipeline(
        [
            ("column transformers", ct),
            ("scler", StandardScaler()),
            ("log_reg", LogisticRegression()),
        ]
)

In [356]:
pipe.fit_transform(cars) # specify Y

ValueError: This LogisticRegression estimator requires y to be passed, but the target y is None.

In [98]:
# one hot encode catagorical variables